# **Imports and Functions**

Notebook created by Antoine Carton-Leclercq, PhD

*Tapper Lab, UMass Chan Medical School, Worcester MA*

In [2]:
import imagej
import os 
import pandas as pd

# Write down Mouse name and folder path
mouse_name = "B2_M4_HC"
mouse_folder = r"E:\B2_M4_HC"

# REPLACE WITH PREFIX CHOSEN WHEN SAVING ACQUIRED IMAGES Default = Image
prefix = 'Image'

Insert the path to the Fiji installation folder

In [5]:


# Initialize ImageJ
ij = imagej.init(r"E:\DATA\QUINT\fiji-latest-win64-jdk\Fiji", headless=False) # Path to FIJI installation FOLDER
print(f"ImageJ2 version: {ij.getVersion()}")

# Define the stitching macro
STITCHING_MACRO = """
#@ String grid_size_x
#@ String grid_size_y
#@ String tile_overlap
#@ String image_path
#@ String image_name
#@ String output_path
#@ String output_filename

run("Grid/Collection stitching",
    "type=[Grid: snake by rows] " +
    "order=[Right & Down                ] " +
    "grid_size_x=" + grid_size_x + " " +
    "grid_size_y=" + grid_size_y + " " +
    "tile_overlap=" + tile_overlap + " " +
    "first_file_index_i=1 " +
    "directory=[" + image_path + "] " +
    "file_names=" + image_name + " " +
    "output_textfile_name=TileConfiguration.txt " +
    "fusion_method=[Linear Blending] " +
    "regression_threshold=0.30 " +
    "max/avg_displacement_threshold=2.50 " +
    "absolute_displacement_threshold=3.50 " +
    "computation_parameters=[Save memory (but be slower)] " +
    "image_output=[Fuse and display]");

// Save the stitched image as TIF
selectWindow("Fused");
saveAs("Tiff", output_path + output_filename);
close();
"""

def stitch_images_with_imagej_macro(temp_folder, output_folder, grid_size_x, grid_size_y, tile_overlap, slice):
    """
    Stitch images using ImageJ Grid/Collection stitching plugin.
    
    Parameters:
    -----------
    temp_folder : str
        Directory containing the input images
    output_folder : str
        Directory where stitched image will be saved
    grid_size_x : int
        Number of columns in the grid
    grid_size_y : int
        Number of rows in the grid
    tile_overlap : int
        Percentage overlap between tiles (typically 10-30%) 
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Convert paths to forward slashes (ImageJ macro compatibility)
    temp_folder_macro = temp_folder.replace('\\', '/')
    output_folder_macro = output_folder.replace('\\', '/')
    
    # Ensure paths end with '/'
    if not temp_folder_macro.endswith('/'):
        temp_folder_macro += '/'
    if not output_folder_macro.endswith('/'):
        output_folder_macro += '/'
    
    # Prepare the macro arguments
    args = {
        "grid_size_x": str(grid_size_x),
        "grid_size_y": str(grid_size_y),
        "tile_overlap": str(tile_overlap),
        "image_path": temp_folder_macro,
        "image_name": f"{prefix}_{slice}"+"_{iiiii}_Z001_Overlay.tif",
        "output_path": output_folder_macro,
        "output_filename": f"{mouse_name}_s{k:04d}.tif"
    }
    
    print(f"Starting stitching with parameters:")
    print(f"  Grid size: {grid_size_x} x {grid_size_y}")
    print(f"  Tile overlap: {tile_overlap}%")
    print(f"  Input directory: {temp_folder_macro}")
    print(f"  Output directory: {output_folder_macro}")
    
    # Run the macro
    ij.py.run_macro(STITCHING_MACRO, args)
    print("Stitching complete!")



The headless flag of imagej.init is deprecated. Use the mode argument instead.


ImageJ2 version: 2.16.0/1.54p


# **Stitching**

In [6]:

# Select all folders in the mouse folder
slide_list = os.listdir(mouse_folder)
# Filter out unwanted folders
slide_list = [s for s in slide_list if "Stitched" not in s and "coordinates" not in s]

# Load stitching coordinates
coordinates = pd.read_csv(os.path.join(mouse_folder,f"{mouse_name}_coordinates.csv"))
# Separate rows and columns
all_rows = coordinates.filter(regex = 'rows')
all_cols = coordinates.filter(regex = 'cols')

k=1
for i,slide in enumerate(slide_list):
    print(f"Processing slide {i+1} of {len(slide_list)}: {slide}")
    slices_list = os.listdir(os.path.join(mouse_folder,slide))
    for j,slice in enumerate(slices_list):
        
        print(f"  Processing slice {j+1} of {len(slices_list)}: {slice}")
        temp_folder = os.path.join(mouse_folder,slide,slice)
        output_folder = os.path.join(mouse_folder,"Stitched")
        
        # Select coordinates for current slide and slice
        col = all_cols.iloc[j,i]
        row = all_rows.iloc[j,i]
        stitch_images_with_imagej_macro(temp_folder, output_folder, grid_size_x=col, grid_size_y=row, tile_overlap=30, slice = slice)
        k+=1


Processing slide 1 of 2: S1_2
  Processing slice 1 of 29: XY01
Starting stitching with parameters:
  Grid size: 14 x 13
  Tile overlap: 30%
  Input directory: E:/B2_M4_HC/S1_2/XY01/
  Output directory: E:/B2_M4_HC/Stitched/
Stitching complete!
  Processing slice 2 of 29: XY02
Starting stitching with parameters:
  Grid size: 14 x 12
  Tile overlap: 30%
  Input directory: E:/B2_M4_HC/S1_2/XY02/
  Output directory: E:/B2_M4_HC/Stitched/


KeyboardInterrupt: 